<a href="https://colab.research.google.com/github/Lui5ito/StatisticsProject/blob/main/Connection_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import pandas as pd

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=4Y5sEe7v9rXVZXov-RaLC-mBo47GtoLlFFPEENTa5iw&tc=vRWrMaV-94r0V1Ip4yPp5-hQxcey9hniPBK8RdfmKpc&cc=9-MN4eW1lFhbPVZGDkRRlKvXhyT7hDZkwTyU3tL4pKc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlW20w2EQJAdGGdVDOVXsuWqWzFyJJdt1n3GK_AabeJ-NKJH_HtRag

Successfully saved authorization token.


In [4]:
ee.Initialize()

In [5]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['longitude', 'latitude', 'datetime',  *list_of_bands]]

    return df

In [7]:
selected_var2 = ["snow_density", "skin_temperature", "total_precipitation", "surface_latent_heat_flux"]
selected_var = ["snow_density"]

In [14]:
image = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY')
dataset_region = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
france = dataset_region.filter(ee.Filter.eq('ADM1_NAME', 'France'))
reg_geom = france.geometry()

In [21]:
image = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
dataset_region = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
region = dataset_region.filter(ee.Filter.Or(ee.Filter.eq('ADM0_NAME', 'France')))
reg = region.geometry()
scale=11132
df_vide=[]
df_vide=pd.DataFrame(df_vide)
g=1
j=0
x=2
z=0
for i in range(12):
    startDate = ee.Date.fromYMD(2000+z,g,1)
    endDate = ee.Date.fromYMD(2000+j,x,1)
    g += 1
    x += 1
    if g>12:
        g=1
        z+=1
    if x>12:
        x=1
        j+=1
    image2=image.select("temperature_2m").filterDate(startDate, endDate)
    image2=image2.getRegion(reg,scale).getInfo()
    print(len(image2))
    occitanie_prec_df=ee_array_to_df(image2,['temperature_2m'])
    occitanie_prec_df['yyyy'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.year
    occitanie_prec_df['mm'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.month
    occitanie_prec_df['dd'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.day
    occitanie_prec_df['date']=occitanie_prec_df['yyyy'].astype('str') + '-'+ occitanie_prec_df['mm'].astype('str')+ '-'+ occitanie_prec_df['dd'].astype('str')
    data=occitanie_prec_df.drop(['datetime', 'yyyy', 'mm', 'dd'], axis=1)
    #moyenne des données des heures par jour
    mean_data=data.groupby(['longitude', 'latitude','date'],as_index=False, sort=False).mean()
    #Construction d'un point geometry comme geopanda mais sans la compléxité du dernier
    mean_data['Point']='Point('+mean_data['longitude'].astype('str') + ' '+ mean_data['latitude'].astype('str')+ ')'
    data_test=mean_data.drop(['longitude', 'latitude'], axis=1)
    df_vide=pd.concat((df_vide,data_test),ignore_index=True, sort=True)

199703
186819
199703
193261
199703
193261
199703
199703
193261
199703
193261
199703


In [22]:
df_vide.head()

,Point,date,temperature_2m
0,Point(9.150041854678939 41.450189603982736),2000-1-1,281.201558
1,Point(9.150041854678939 41.450189603982736),2000-1-2,281.058940
2,Point(9.150041854678939 41.450189603982736),2000-1-3,282.557199
3,Point(9.150041854678939 41.450189603982736),2000-1-4,283.243879
4,Point(9.150041854678939 41.450189603982736),2000-1-5,284.472150


In [26]:
image = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
dataset_region = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
region = dataset_region.filter(ee.Filter.Or(ee.Filter.eq('ADM0_NAME', 'France')))
reg = region.geometry()
scale=11132
df_vide=[]
df_vide=pd.DataFrame(df_vide)

# Date de début initiale (02 janvier 1950)
start_year = 1950
start_month = 1
start_day = 2

# Date de fin souhaitée (14 octobre 2023)
end_year = 2023
end_month = 10
end_day = 14

# Crée une boucle pour les années
for year in range(start_year, end_year + 1):
    # Détermine le mois de début et de fin pour cette année
    if year == start_year:
        start_month_range = start_month
    else:
        start_month_range = 1

    if year == end_year:
        end_month_range = end_month
    else:
        end_month_range = 12

    # Boucle à travers les mois
    for month in range(start_month_range, end_month_range + 1):
        if end_month_range != 12:
          startDate = ee.Date.fromYMD(year, month, 1)
          endDate = ee.Date.fromYMD(year, month, 1)
          image2=image.select("temperature_2m").filterDate(startDate, endDate)
          image2=image2.getRegion(reg,scale).getInfo()
          print(len(image2))
          occitanie_prec_df=ee_array_to_df(image2,['temperature_2m'])
          occitanie_prec_df['yyyy'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.year
          occitanie_prec_df['mm'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.month
          occitanie_prec_df['dd'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.day
          occitanie_prec_df['date']=occitanie_prec_df['yyyy'].astype('str') + '-'+ occitanie_prec_df['mm'].astype('str')+ '-'+ occitanie_prec_df['dd'].astype('str')
          data=occitanie_prec_df.drop(['datetime', 'yyyy', 'mm', 'dd'], axis=1)
          #moyenne des données des heures par jour
          mean_data=data.groupby(['longitude', 'latitude','date'],as_index=False, sort=False).mean()
          #Construction d'un point geometry comme geopanda mais sans la compléxité du dernier
          mean_data['Point']='Point('+mean_data['longitude'].astype('str') + ' '+ mean_data['latitude'].astype('str')+ ')'
          data_test=mean_data.drop(['longitude', 'latitude'], axis=1)
          df_vide=pd.concat((df_vide,data_test),ignore_index=True, sort=True)
        else:
          startDate = ee.Date.fromYMD(year, month, 1)
          endDate = ee.Date.fromYMD(year, month, 1)
          image2=image.select("temperature_2m").filterDate(startDate, endDate)
          image2=image2.getRegion(reg,scale).getInfo()
          print(len(image2))
          occitanie_prec_df=ee_array_to_df(image2,['temperature_2m'])
          occitanie_prec_df['yyyy'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.year
          occitanie_prec_df['mm'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.month
          occitanie_prec_df['dd'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.day
          occitanie_prec_df['date']=occitanie_prec_df['yyyy'].astype('str') + '-'+ occitanie_prec_df['mm'].astype('str')+ '-'+ occitanie_prec_df['dd'].astype('str')
          data=occitanie_prec_df.drop(['datetime', 'yyyy', 'mm', 'dd'], axis=1)
          #moyenne des données des heures par jour
          mean_data=data.groupby(['longitude', 'latitude','date'],as_index=False, sort=False).mean()
          #Construction d'un point geometry comme geopanda mais sans la compléxité du dernier
          mean_data['Point']='Point('+mean_data['longitude'].astype('str') + ' '+ mean_data['latitude'].astype('str')+ ')'
          data_test=mean_data.drop(['longitude', 'latitude'], axis=1)
          df_vide=pd.concat((df_vide,data_test),ignore_index=True, sort=True)

EEException: ignored